Glenys Code

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_set = pd.read_csv("/Users\strua\Downloads\Train.csv")

train_set_copy = train_set

y_train = train_set_copy[["target"]]

X_train = train_set_copy

X_train.drop('target', axis=1, inplace=True) 
X_train.drop('id', axis=1, inplace=True)

for col in ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']:
    X_train = pd.concat([X_train,pd.get_dummies(X_train[col], prefix=col)],axis=1)
    X_train.drop(col, axis = 1, inplace = True)
    
test_set = pd.read_csv("/Users\strua\Downloads\Test.csv")

test_set_copy = test_set

X_test = test_set_copy                       
X_test.drop('id', axis = 1, inplace = True)  
X_test.head()

for col in ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']:
    X_test = pd.concat([X_test,pd.get_dummies(X_test[col], prefix=col)],axis=1)
    X_test.drop(col, axis = 1, inplace = True)
    
cat10_train = X_train['cat10']

cat10_test = X_test['cat10']

after_merge = test_set_copy.iloc[0:1, :]

for col in ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']:
    after_merge = pd.concat([after_merge,pd.get_dummies(after_merge[col], prefix=col)],axis=1)
    after_merge.drop(col, axis = 1, inplace = True)
    
cat10_combined = pd.concat([cat10_train, cat10_test])

cat10_encoded = pd.get_dummies(cat10_combined, prefix='cat10') 

new_cat10_train = cat10_encoded.iloc[0:300000,:]   
new_cat10_test = cat10_encoded.iloc[300000:,:]

X_train.drop('cat10', axis = 1, inplace = True)

X_train_final = pd.concat([X_train, new_cat10_train], axis = 1)

X_test.drop('cat10', axis = 1, inplace = True)

X_test_final = pd.concat([X_test, new_cat10_test], axis = 1)

Random Forest Classifier

A random forest classifier is an estimator that fit numerous decission trees on subset of the training data and then averages  them to improve its predictive accuracy. The number of these trees can we edited to give a better prediction.

Firstly we start by importing the classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

We will run this classifier with n_estimators = 100, this means that there will be 100 decission trees that this classifier will base its results off. Our random_state ensures this classifer will remain consistent in any future runs. 

Once we make our model we will fit it on our training data.

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rnd_clf.fit(X_train_final, np.ravel(y_train))

Now we want to see how well the predictive quality of our model is for this we will import a function called accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score

Since we don't have actual data for our test data we will see how well the model will predict the training data since we have that information available to us. 

In [ ]:
yt_pred_rf = rnd_clf.predict(X_train_final)
accuracy_score( y_train,yt_pred_rf) # returns 0.9999833333333333

This accuracy score is very high which could mean that our model might not be fit for use as it possibly will overfit our test data. We will save our training predictions incase we want to see the areas where it has not fitted

In [ ]:
np.savetxt('RandomForestTrainingPreds.csv',yt_pred_rf,delimiter=",")

Now we will run our model on the test data.

In [ ]:
y_pred_rf = rnd_clf.predict(X_test_final)

and then once again save our predictions of our test data

In [ ]:
np.savetxt("RandomForestTestPreds.csv",y_pred_rf,delimiter=',')

Bagging Classifier

A bagging classifier is a classifier that fits its data on subsets of the training data and the

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bagT_clf = BaggingClassifier(
   DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bagT_clf.fit(X_train_final, np.ravel(y_train))

In [ ]:
yt_pred_bagT = bagT_clf.predict(X_train_final)
accuracy_score(y_train,yt_pred_bagT) #returns 0.8351866666666666

In [ ]:
np.savetxt("BaggingTreeTrainingPreds.csv",yt_pred_bagT,delimiter=',')

In [ ]:
y_pred_bagT = bagT_clf.predict(X_test_final)

In [ ]:
np.savetxt("BaggingTreePreds.csv",y_pred_bagT,delimiter=',')

In [ ]:
bagF_clf = BaggingClassifier(
    RandomForestClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bagF_clf.fit(X_train_final, np.ravel(y_train))

In [ ]:
yt_pred_bagF = bagF_clf.predict(X_train_final)
accuracy_score(y_train,yt_pred_bagF) #returns 0.8257533333333333

In [ ]:
np.savetxt("BaggingForestTrainingPreds.csv",yt_pred_bagF,delimiter=',')

In [ ]:
y_pred_bagF = bagF_clf.predict(X_test_final)

In [ ]:
np.savetxt("BaggingForestPreds.csv",y_pred_bagF,delimiter=',')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train_final, np.ravel(y_train))

In [ ]:
yt_pred_Ada = ada_clf.predict(X_train_final)
accuracy_score(y_train,yt_pred_Ada) #returns 0.8434366666666666

In [ ]:
np.savetxt("AdaBoostTrainingPreds.csv",yt_pred_Ada,delimiter=',')

In [ ]:
y_pred_Ada = ada_clf.predict(X_test_final)

In [ ]:
np.savetxt("AdaBoostPreds.csv",y_pred_Ada,delimiter=',')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
Grd_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
        max_depth=1, random_state=42)
Grd_clf.fit(X_train_final,np.ravel(y_train))

In [ ]:
yt_pred_Grad = Grd_clf.predict(X_train_final)
accuracy_score(y_train,yt_pred_Grad) #return 0.8439566666666667

In [ ]:
np.savetxt("GradientBoostTrainingPreds.csv",yt_pred_Grad,delimiter=',')

In [ ]:
y_pred_Grad = Grd_clf.predict(X_test_final)

In [ ]:
np.savetxt("GradientBoostPreds.csv",y_pred_Grad,delimiter=',')